In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from starter_code.utils import load_case, load_volume
from starter_code.visualize import visualize

import tensorflow as tf
import os
import sys
 
import numpy as np
 
from tqdm import tqdm
from itertools import chain
 
from skimage.io import imread, imshow

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import gc
import matplotlib.pyplot as plt
import nibabel as nib

In [0]:
!unzip 'drive/My Drive/data.zip' # this folder contains all the train and test data
!unzip 'drive/My Drive/starter_code.zip' # this folder contains all the code to manipulate the data

In [0]:
!mkdir data # we move data to a folder stored on the disk
!mv -f case** data

In [0]:
IMG_WIDTH = 512
IMG_HEIGHT = 512
IMG_CHANNELS = 1

In [0]:
# This function build a model that got inspired by the U-Net architecture.

def build_model():
    inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(s)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(p3)
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(p4)
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(c5)

    u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(u6)
    c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(3, (1, 1), activation='softmax')(c9)

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    return model

In [0]:
# Here we build the model and compile it

model = build_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [0]:
# Data preprocessing and training

for o in range(0, 210, 10):
  if o != 0:
    model = load_model("drive/My Drive/model.h5")

  X = None
  Y = None

  for i in range(o, o + 5):
      volume, segmentation = load_case(i)
      try:
          data_v = volume.get_fdata()          # we extract corresponding numpy array data
          data_s = segmentation.get_fdata()
          if X is None:
              X = data_v
              Y = data_s
          else:
              X = np.concatenate([X, data_v])
              Y = np.concatenate([Y, data_s])
      except:
          print(i)
      if i % 5 == 0:
          print(i)

      X = X.reshape((X.shape[0], 512, 512, 1))
      # One hot encoding of Y
      Y = to_categorical(Y, 3)

      checkpoint_path = "drive/My Drive/cp1.ckpt"
      checkpoint_dir = os.path.dirname(checkpoint_path)
      
      # Create checkpoint callback
      cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                      save_weights_only=True,
                                                      verbose=1)
      
      callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='./logs', profile_batch=100000000),
        cp_callback 
      ]

      # We fit the model to the data
      history = model.fit(X, Y, validation_split=0.15, batch_size=32, epochs=20, verbose=1)
      model.save("drive/My Drive/model.h5")

      # Plot training & validation accuracy values
      plt.plot(history.history['acc'])
      plt.plot(history.history['val_acc'])
      plt.title('Model accuracy')
      plt.ylabel('Accuracy')
      plt.xlabel('Epoch')
      plt.show()

      # Plot training & validation loss values
      plt.plot(history.history['loss'])
      plt.plot(history.history['val_loss'])
      plt.title('Model loss')
      plt.ylabel('Loss')
      plt.xlabel('Epoch')
      plt.show()

In [0]:
# This function applies a one hot decode to a given prediction

def onehot_decode(X):
  res = np.empty((X.shape[0], X.shape[1], X.shape[2]))
  for x in range(X.shape[0]):
    for y in range(X.shape[1]):
        res[x][y] = np.argmax(X[x][y], axis=1)
  return res

In [0]:
model = load_model("drive/My Drive/model.h5")  # We load our saved model

for i in range(200, 300):
  volume = load_volume(i)  # We load test image
  try:
    affine = volume.affine
    X = volume.get_fdata()
    X = X.reshape((X.shape[0], 512, 512, 1))
    X_pred = model.predict(X)   # We feed it to our neural network to get the prediction

    res = onehot_decode(X_pred)  # One hot decode + reconstruction of a new NIFTI Image
    img = nib.Nifti1Image(res, affine)
    os.mkdir("drive/My Drive/results/" + "case_{:05d}".format(i))

    os.system("touch drive/My Drive/results/" + "case_{:05d}".format(i) +"/segmentation.nii")
    img.to_filename("drive/My Drive/results/" + "case_{:05d}".format(i) +"/segmentation.nii")
  except Exception as e:
    print("issue {} {}".format(e, i))
  
  if i % 5 == 0:
    print(i)